In [ ]:
#| echo: false
#| output: false

import Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
#| echo: false
#| output: false

using JuMP
using HiGHS
using Plots
using StatsPlots
using Measures
using LaTeXStrings
using DataFrames
using CSV
using MarkdownTables

# Review and Questions

## Last Class

- Resource allocation problem as a linear program
- Worked land allocation example

## Questions?

:::: {.columns .center}
::: {.column width="40%"}
![](images/vsrikrish-poll.png){fig-alt="Poll Everywhere QR Code" fig-align="center" width="100%"}
:::
::: {.column width=60%}
**Text**: VSRIKRISH to 22333

**URL**: [https://pollev.com/vsrikrish](https://pollev.com/vsrikrish)
<br><br>
[See Results](https://www.polleverywhere.com/multiple_choice_polls/qKeig0hHShZYHAOmvZtZm?preview=true&controls=none){preview-link="true"}
:::
::::



# Electric Power System Decision Problems

## Overview of Electric Power Systems

![Power Systems Schematic](https://upload.wikimedia.org/wikipedia/commons/4/41/Electricity_grid_simple-_North_America.svg)

::: {.caption}
Source: [Wikipedia](https://en.wikipedia.org/wiki/Electric_power_transmission)
:::

## Decisions Problems for Power Systems

![Decision Problems for Power Systems by Time Scale](figures/elec-decision-problems.svg)

::: {.caption}
Adapted from Perez-Arriaga, Ignacio J., Hugh Rudnick, and Michel Rivier (2009)
:::

## Electricity Generation by Source

![Electricity Generation in 2022 by Source](https://www.eia.gov/energyexplained/electricity/images/outlet-graph-large.jpg)

::: {.caption}
Source: U.S. Energy Information Administration
:::

# Generating Capacity Expansion

## Capacity Expansion

**Capacity expansion** involves adding resources to generate or transmit electricity to meet anticipated demand (*load*) in the future.

Typical objective: Minimize cost

But other constraints may apply, *e.g.* reducing CO~2~ emissions or increasing fuel diversity.


# Simple Capacity Expansion Example

## Plant Types

In general, we have many fuel options:

- Gas (combined cycle or simple cycle);
- Coal;
- Nuclear;
- Renewables (wind, solar, hydro, geothermal)

## Simplified Example: Generators 


In [ ]:
gens = DataFrame(CSV.File("data/capacity_expansion/generators.csv"))
gens_display = rename(gens, Symbol.([:Generator, :"Fixed Cost (\\\$)", :"Variable Cost (\\\$/MW)"]))
markdown_table(gens_display[1:end-2, :])

## Simplified Example: Demand


In [ ]:
#| label: fig-demand
#| fig-cap: Demand for 2020 in NYISO Zone C
#| fig-align: center

NY_demand = DataFrame(CSV.File("data/capacity_expansion/2020_hourly_load_NY.csv"))
rename!(NY_demand, :"Time Stamp" => :Date)
demand = NY_demand[:, [:Date, :C]]
rename!(demand, :C => :Demand)
@df demand plot(:Date, :Demand, xlabel="Date", ylabel="Demand (MWh)", label=:false, tickfontsize=14, guidefontsize=14, xrot=45, bottommargin=20mm, leftmargin=7mm)
plot!(size=(1200, 550))

## Simplified Example: Load Duration Curve


In [ ]:
#| label: fig-ldc
#| fig-cap: Load Duration Curve for 2020 in NYISO Zone C
#| fig-align: center

plot(sort(demand.Demand, rev=true), linewidth=3, label=:false, leftmargin=8mm, bottommargin=20mm, tickfontsize=14, guidefontsize=14)
xaxis!("Hours")
yaxis!("Demand (MWh)")
plot!(size=(1200, 550))

## Capacity Expansion: Goal

We want to find the installed capacity of each technology that meets demand at all hours at minimal *total cost*.

::: {.fragment .fade-in}
*Although...*

::: {.incremental}
- For some hours with extreme load, is it necessarily worth it to build new generation to meet those peaks?
- Instead, assign a *high* cost $NSECost$ to **non-served energy** (NSE). In this case, let's set $NSECost = \$9000$/MWh
:::
:::

## Decision Variables 

What are our variables?

::: {.fragment .fade-in}
| Variable | Meaning |
|:--:|:-------------------:|
| $x_g$ | installed capacity (MW) from each generator type $g \in \mathcal{G}$ |
| $y_{g,t}$ | production (MW) from each generator type $g$ in hour $t \in \mathcal{T}$ |
| $NSE_t$ | non-served energy (MWh) in hour $t \in \mathcal{T}$ |
:::

## Capacity Expansion Objective

$$
\begin{align}
\min_{x, y, NSE} Z &= {\color{red}\text{investment cost} } + {\color{blue}\text{operating cost} } \\
&= {\color{red} \sum_{g \in \mathcal{G}} \text{FixedCost}_g x_g} + {\color{blue} \sum_{t \in \mathcal{T}} \sum_{g \in \mathcal{G}}  \text{VarCost}_g y_{g,t} + \sum_{t \in \mathcal{T}} \text{NSECost} NSE_t}
\end{align}
$$

## What Are Our Constraints?

:::: {.columns .center}
::: {.column width="40%"}
![](images/vsrikrish-poll.png){fig-alt="Poll Everywhere QR Code" fig-align="center" width="100%"}
:::
::: {.column width=60%}
**Text**: VSRIKRISH to 22333

**URL**: [https://pollev.com/vsrikrish](https://pollev.com/vsrikrish)
<br><br>
[See Results](https://www.polleverywhere.com/multiple_choice_polls/qKeig0hHShZYHAOmvZtZm?preview=true&controls=none){preview-link="true"}
:::
::::



## Capacity Expansion Constraints

- **Demand**: Sum of generated energy and non-served energy must equal demand $d_t$.
- **Capacity**: Generator types cannot produce more electricity than their installed capacity.
- **Non-negativity**: All decision variables must be non-negative.

## Problem Formulation

$$
\begin{align}
\min_{x, y, NSE} \quad & \sum_{g \in \mathcal{G}} \text{FixedCost}_g x_g + \sum_{t \in \mathcal{T}} \sum_{g \in \mathcal{G}} \text{VarCost}_gy_{g,t} & \\
& \quad + \sum_{t \in \mathcal{T}} \text{NSECost} NSE_t & \\[0.5em]
\text {subject to:} \quad & \sum_{g \in \mathcal{G}} y_{g,t} + NSE_t = d_t & \forall t \in \mathcal{T} \\[0.5em]
& y_{g,t} \leq x_g & \forall g \in {G},  \forall t \in \mathcal{T} \\[0.5em]
& x_g, y_{g,t}, NSE_t \geq 0 & \forall g \in {G},  \forall t \in \mathcal{T}
\end{align}
$$

## Capacity Expansion is an LP

- **Linearity**: costs assumed to scale linearly;
- **Divisible**: we model total installed capacity, not number of individual units
- **Certainty**: no uncertainty about renewables.

Real problems can get much more complex, particularly if we try to model making decisions under renewable or load uncertainty.

## What Does This Problem Neglect?

1. **Discrete decisions**: is a plant on or off (this is called *unit commitment*)?
2. **Intertemporal constraints**: Power plants can't just "ramp" from producing low levels of power to high levels of power; there are real engineering limits.
3. **Transmission**: We can generate all the power we want, but what if we can't get it to where the demand is?

## Capacity Expansion Example Solution


In [ ]:
#| output: false

# define sets
G = 1:nrow(gens[1:end-2, :])
T = 1:nrow(demand)
NSECost = 9000

gencap = Model(HiGHS.Optimizer)
# define variables
@variables(gencap, begin
    x[g in G] >= 0
    y[g in G, t in T] >= 0
    NSE[t in T] >= 0
end)
@objective(gencap, Min, 
    sum(gens[G, :FixedCost] .* x) + sum(gens[G, :VarCost] .* sum(y[:, t] for t in T)) + NSECost * sum(NSE)
)
@constraint(gencap, load[t in T], sum(y[:, t]) + NSE[t] == demand.Demand[t])
@constraint(gencap, availability[g in G, t in T], y[g, t] <= x[g])
optimize!(gencap)

In [ ]:
generation = zeros(size(G,1))
for i in 1:size(G,1) 
    generation[i] = sum(value.(y)[G[i],:].data) 
end
MWh_share = generation./sum(demand.Demand).*100
cap_share = value.(x).data ./ maximum(demand.Demand) .* 100
results = DataFrame(
    Resource = gens.Plant[G], 
    Capacity = value.(x).data,
    Percent_Cap = cap_share,
    Generated = generation/1000,
    Percent_Gen = MWh_share
)
rename!(results, 
    [:Capacity => :"Capacity (MW)", 
    :Percent_Cap => :"Percent_Cap (%)", 
    :Generated => "Generated (GWh)",
    :Percent_Gen => "Percent_Gen (%)"])
results[:, 2:end] = round.(results[:, 2:end], digits=1)

NSE_MW = maximum(value.(NSE).data) 
NSE_MWh = sum(value.(NSE).data)
push!(results, ["NSE" NSE_MW NSE_MW/maximum(demand.Demand)*100 NSE_MWh/1000 NSE_MWh/sum(demand.Demand)*100])

results[:, 2:end] = round.(results[:, 2:end], digits=1)
markdown_table(results)

## When Do Generators Operate?


In [ ]:
#| label: fig-results
#| fig-cap: Results of Generating Capacity Expansion Example

areaplot(value.(y).data', 
    label=permutedims(gens.Plant), 
    xlabel = "Hour", 
    ylabel ="Generated Electricity (MWh)", 
    color_palette=:seaborn_colorblind, 
    size=(700, 700), 
    left_margin=5mm, 
    bottom_margin=5mm,
    grid=:false,
    legend = :topleft, ylim=(0, 3200),
    legendfontsize=14, tickfontsize=14, 
    guidefontsize=14)
plot!(size=(1200, 550))

## What Else Could We Add?

:::: {.columns .center}
::: {.column width="40%"}
![](images/vsrikrish-poll.png){fig-alt="Poll Everywhere QR Code" fig-align="center" width="100%"}
:::
::: {.column width=60%}
**Text**: VSRIKRISH to 22333

**URL**: [https://pollev.com/vsrikrish](https://pollev.com/vsrikrish)
<br><br>
[See Results](https://www.polleverywhere.com/multiple_choice_polls/qKeig0hHShZYHAOmvZtZm?preview=true&controls=none){preview-link="true"}
:::
::::



# Key Takeaways

## Key Takeaways

- Capacity Expansion is a foundational power systems decision problem.
- Is an LP with some basic assumptions.
- We looked at a "greenfield" example: no existing plants.
- Decision problem becomes more complex with renewables (HW4) or "brownfield" (expanding existing fleet).

# Upcoming Schedule

## Next Classes

**Monday**: Economic Dispatch
**Wednesday/Friday**: Air Pollution

## Assessments

- Lab 3 due **tonight** at 9pm.